In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import zipfile
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm
import time
import copy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
data_dir = '/content/drive/MyDrive/2024-2 DScover 메인프로젝트 D조/Data'

In [ ]:
num_classes = 4
batch_size = 32
num_epochs = 50
learning_rate = 1e-4

In [ ]:
# 데이터 전처리 (학습 시 데이터 증강 포함)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet 평균
                             [0.229, 0.224, 0.225])  # ImageNet 표준편차
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# 데이터셋 로드
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}

# 데이터로더 생성
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=4)
               for x in ['train', 'valid', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
class_names = image_datasets['train'].classes

print(f'Classes: {class_names}')

Classes: ['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


In [ ]:
# 1. SE Block 정의
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(in_channels, in_channels // reduction)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size, channels, _, _ = x.size()
        se = self.global_avg_pool(x).view(batch_size, channels)
        se = self.fc1(se)
        se = F.relu(se)
        se = self.fc2(se)
        se = self.sigmoid(se).view(batch_size, channels, 1, 1)
        return x * se

class SEBasicBlock(BasicBlock):
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None, reduction=16):
        super(SEBasicBlock, self).__init__(inplanes, planes, stride, downsample, groups, base_width, dilation, norm_layer)

        # SE Block 추가
        self.se_block = SEBlock(planes, reduction)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # SE Block 적용
        out = self.se_block(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


def resnet18_with_se(pretrained=False, progress=True, num_classes=1000):
    model = ResNet(SEBasicBlock, [2, 2, 2, 2], num_classes=num_classes)
    if pretrained:
        pretrained_weights_url = 'https://download.pytorch.org/models/resnet18-f37072fd.pth'
        pretrained_dict = torch.hub.load_state_dict_from_url(pretrained_weights_url, progress=progress)

        # 모델의 현재 가중치와 비교
        model_dict = model.state_dict()

        # FC 레이어 제외하고 매칭되는 가중치만 로드
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and not k.startswith('fc')}
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)
    return model


# 4. 모델 생성 및 파인튜닝 설정
model = resnet18_with_se(pretrained=True, num_classes=num_classes)

# Feature Extractor로 사용하기 위해 파라미터 고정
for param in model.parameters():
    param.requires_grad = False

# Fully Connected Layer 수정
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Fully Connected Layer만 학습 가능하도록 설정
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

# 5. Optimizer와 Loss 설정
params_to_update = [param for param in model.parameters() if param.requires_grad]
optimizer = optim.Adam(params_to_update, lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 6. 훈련 및 검증 함수 정의
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, device, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # 학습 단계
        model.train()
        total_loss = 0.0
        running_corrects = 0
 ㅈ
        for batch in tqdm(dataloaders['train'], desc="Training"):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)  # (batch_size, num_classes)
            loss = criterion(outputs, labels)  # 크로스엔트로피 손실

            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        train_epoch_loss = total_loss / dataset_sizes['train']
        train_epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f"Epoch {epoch+1}: Train Loss = {train_epoch_loss:.4f}, Train Acc = {train_epoch_acc:.4f}")

        # 검증 단계
        model.eval()
        eval_loss = 0.0
        eval_corrects = 0

        with torch.no_grad():
            for batch in tqdm(dataloaders['valid'], desc="Validation"):
                inputs, labels = batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                eval_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                eval_corrects += torch.sum(preds == labels.data)

        eval_epoch_loss = eval_loss / dataset_sizes['valid']
        eval_epoch_acc = eval_corrects.double() / dataset_sizes['valid']
        print(f"Epoch {epoch+1}: Eval Loss = {eval_epoch_loss:.4f}, Eval Acc = {eval_epoch_acc:.4f}")

        # 최고 성능 모델 저장
        if eval_epoch_acc > best_acc:
            best_acc = eval_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best validation Acc: {best_acc:.4f}')

    # 최고 모델 가중치 로드
    model.load_state_dict(best_model_wts)
    return model

# 7. 모델 훈련 실행
model = train_model(model, criterion, optimizer, dataloaders, dataset_sizes, device, num_epochs=num_epochs)

# 8. 테스트 데이터셋에서 모델 평가
def evaluate_model(model, dataloader, dataset_size, device):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Testing"):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / dataset_size
    print(f'Test Accuracy: {accuracy:.4f}')

evaluate_model(model, dataloaders['test'], dataset_sizes['test'], device)

Epoch 1/50
----------


Training:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1: Train Loss = 1.4295, Train Acc = 0.2773


Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1: Eval Loss = 1.4741, Eval Acc = 0.2639

Epoch 2/50
----------


...

Epoch 50: Train Loss = 0.8812, Train Acc = 0.6199


Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 50: Eval Loss = 0.9965, Eval Acc = 0.5139

Training complete in 2m 34s
Best validation Acc: 0.5139


Testing:   0%|          | 0/10 [00:00<?, ?it/s]

Test Accuracy: 0.6317
